One of the leading retail stores in the US, Walmart, would like to predict the sales and demand accurately. There are certain events and holidays which impact sales on each day. There are sales data available for 45 stores of Walmart. The business is facing a challenge due to unforeseen demands and runs out of stock some times, due to the inappropriate machine learning algorithm. An ideal ML algorithm will predict demand accurately and ingest factors like economic conditions including CPI, Unemployment Index, etc.

Walmart runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of all, which are the Super Bowl, Labour Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks. Part of the challenge presented by this competition is modeling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data. Historical sales data for 45 Walmart stores located in different regions are available.

About this file: This is the historical data that covers sales from 2010-02-05 to 2012-11-01, in the file Walmart.csv. 

Within this file you will find the following fields:
- Store: the store number
- Date: the week of sales
- Weekly_sales: sales for the given store
- Holiday_Flag: whether the week is a special holiday week (1) and a non-holiday week (0)
- Temperature: temperature on the day of sale
- Fuel_Price: Cost of fuel in te region
- CPI: prevailing consumer price index
- Unemployment: prevailing unemployment rate
- Holiday Events:

        Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
        Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
        Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
        Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

In [70]:
-- Compares the different types of holidays and non-holiday weeks with total weekly sales, and average weekly sales to analyze their sales based on holidays
SELECT CASE 
		WHEN DATE IN (
				'2010-02-12'
				,'2011-02-11'
				,'2012-02-10'
				,'2013-02-08'
				)
			THEN 'Super Bowl'
		WHEN DATE IN (
				'2010-09-10'
				,'2011-09-09'
				,'2012-09-07'
				,'2013-09-06'
				)
			THEN 'Labor Day'
		WHEN DATE IN (
				'2010-11-26'
				,'2011-11-25'
				,'2012-11-23'
				,'2013-11-29'
				)
			THEN 'Thanksgiving'
		WHEN DATE IN (
				'2010-12-31'
				,'2011-12-30'
				,'2012-12-28'
				,'2013-12-27'
				)
			THEN 'Christmas'
		ELSE 'Non-Holiday'
		END AS Holiday_Name
	,ROUND(SUM(Weekly_Sales), 2) AS Total_Weekly_Sales
	,ROUND(AVG(Weekly_Sales), 2) AS Avg_Weekly_Sales
FROM 'Walmart.csv'
GROUP BY Holiday_Name
ORDER BY Total_Weekly_Sales DESC;

,Holiday_Name,Total_Weekly_Sales,Avg_Weekly_Sales
0,Non-Holiday,6.231919e+09,1041256.38
1,Super Bowl,1.456823e+08,1079127.99
2,Labor Day,1.407277e+08,1042427.29
3,Thanksgiving,1.324146e+08,1471273.43
4,Christmas,8.647498e+07,960833.11


In [68]:
-- Compares the different type of holidays with the average CPI, average unemployment, and total sales to identify economic factors that are impacting holiday total sales
SELECT CASE 
		WHEN DATE IN (
				'2010-02-12'
				,'2011-02-11'
				,'2012-02-10'
				,'2013-02-08'
				)
			THEN 'Super Bowl'
		WHEN DATE IN (
				'2010-09-10'
				,'2011-09-09'
				,'2012-09-07'
				,'2013-09-06'
				)
			THEN 'Labor Day'
		WHEN DATE IN (
				'2010-11-26'
				,'2011-11-25'
				,'2012-11-23'
				,'2013-11-29'
				)
			THEN 'Thanksgiving'
		WHEN DATE IN (
				'2010-12-31'
				,'2011-12-30'
				,'2012-12-28'
				,'2013-12-27'
				)
			THEN 'Christmas'
		ELSE 'Non-Holiday'
		END AS Holiday_Name
	,ROUND(AVG(CPI), 2) AS Avg_CPI
	,ROUND(AVG(Unemployment), 2) AS Avg_Unemployment
	,ROUND(SUM(Weekly_Sales), 2) AS Total_Sales
FROM 'Walmart.csv'
WHERE Holiday_Flag = 1
GROUP BY Holiday_Name
ORDER BY Total_Sales DESC;

,Holiday_Name,Avg_CPI,Avg_Unemployment,Total_Sales
0,Super Bowl,170.63,8.14,1.456823e+08
1,Labor Day,172.11,7.92,1.407277e+08
2,Thanksgiving,170.98,8.14,1.324146e+08
3,Christmas,171.26,8.14,8.647498e+07


In [73]:
-- Takes the weekly sales and the previous weekly sales to create a sales change which will allow Walamrt to analyze stores whose sales have declined significantly
WITH Weekly_Store_Sales
AS (
	SELECT Store
		,Date
		,Weekly_Sales
		,LAG(Weekly_Sales) OVER (
			PARTITION BY Store ORDER BY DATE
			) AS Previous_Week_Sales
	FROM 'Walmart.csv'
	)
	,Sales_Decline
AS (
	SELECT Store
		,Date
		,Weekly_Sales
		,Previous_Week_Sales
		,ROUND((Weekly_Sales - Previous_Week_Sales), 2) AS Sales_Change
	FROM Weekly_Store_Sales
	WHERE Weekly_Sales < Previous_Week_Sales
	)
SELECT Store
	,Date
	,Weekly_Sales
	,Sales_Change
FROM Sales_Decline
ORDER BY Sales_Change
	,Date;

,Store,Date,Weekly_Sales,Sales_Change
0,14,2010-12-31 00:00:00+00:00,1623716.46,-2194969.99
1,10,2010-12-31 00:00:00+00:00,1707298.14,-2041759.55
2,20,2010-12-31 00:00:00+00:00,1799737.79,-1966949.64
3,13,2010-12-31 00:00:00+00:00,1675292.00,-1920611.20
4,4,2010-12-31 00:00:00+00:00,1794868.74,-1731844.65
...,...,...,...,...
3253,37,2010-09-10 00:00:00+00:00,510296.07,-131.46
3254,36,2012-06-01 00:00:00+00:00,306005.53,-92.64
3255,36,2011-11-25 00:00:00+00:00,332811.55,-90.39
3256,33,2011-04-29 00:00:00+00:00,248561.86,-41.44


In [74]:
-- Correlation between weekly sales and multiple factors (temperature, fuel price, CPI, unemployment, and holidays) that could influence Walmart sales
WITH Weekly_Correlation_Factors
AS (
	SELECT Store
		,DATE
		,Weekly_Sales
		,Temperature
		,Fuel_Price
		,CPI
		,Unemployment
		,Holiday_Flag
	FROM 'Walmart.csv'
	)
	,Correlation_Temp
AS (
	SELECT ROUND(CORR(Weekly_Sales, Temperature), 2) AS Correlation_Temp_Sales
	FROM Weekly_Correlation_Factors
	)
	,Correlation_Fuel
AS (
	SELECT ROUND(CORR(Weekly_Sales, Fuel_Price), 2) AS Correlation_Fuel_Sales
	FROM Weekly_Correlation_Factors
	)
	,Correlation_CPI
AS (
	SELECT ROUND(CORR(Weekly_Sales, CPI), 2) AS Correlation_CPI_Sales
	FROM Weekly_Correlation_Factors
	)
	,Correlation_Unemployment
AS (
	SELECT ROUND(CORR(Weekly_Sales, Unemployment), 2) AS Correlation_Unemployment_Sales
	FROM Weekly_Correlation_Factors
	)
	,Correlation_Holiday
AS (
	SELECT ROUND(CORR(Weekly_Sales, Holiday_Flag), 2) AS Correlation_Holiday_Sales
	FROM Weekly_Correlation_Factors
	)
SELECT ct.Correlation_Temp_Sales
	,cf.Correlation_Fuel_Sales
	,cc.Correlation_CPI_Sales
	,cu.Correlation_Unemployment_Sales
	,ch.Correlation_Holiday_Sales
FROM Correlation_Temp AS ct
CROSS JOIN Correlation_Fuel AS cf
CROSS JOIN Correlation_CPI AS cc
CROSS JOIN Correlation_Unemployment AS cu
CROSS JOIN Correlation_Holiday AS ch;

,Correlation_Temp_Sales,Correlation_Fuel_Sales,Correlation_CPI_Sales,Correlation_Unemployment_Sales,Correlation_Holiday_Sales
0,-0.06,0.01,-0.07,-0.11,0.04
